# Hello GERD

Getting started with GERD

In [ ]:
# install dependencies
!pip install poetry
!poetry install --extras cpu --extras gguf

Installing dependencies from lock file

Package operations: 1 install, 33 updates, 17 removals

  - Removing aiofiles (23.2.1): Pending...
  - Removing aiofiles (23.2.1): Removing...
  - Removing aiofiles (23.2.1)
  - Removing ffmpy (0.4.0): Pending...
  - Removing ffmpy (0.4.0): Removing...
  - Removing ffmpy (0.4.0)
  - Removing gradio (5.5.0): Pending...
  - Removing gradio (5.5.0): Removing...
  - Removing gradio (5.5.0)
  - Removing gradio-client (1.4.2): Pending...
  - Removing gradio-client (1.4.2): Removing...
  - Removing gradio-client (1.4.2)
  - Removing markdown-it-py (3.0.0): Pending...
  - Removing markdown-it-py (3.0.0): Removing...
  - Removing markdown-it-py (3.0.0)
  - Removing mdurl (0.1.2): Pending...
  - Removing mdurl (0.1.2): Removing...
  - Removing mdurl (0.1.2)
  - Removing pandas (2.2.3): Pending...
  - Removing pandas (2.2.3): Removing...
  - Removing pandas (2.2.3)
  - Removing pydub (0.25.1): Pending...
  - Removing pydub (0.25.1): Removing...
  - Removing

In [3]:
from gerd.config import load_gen_config
from gerd.gen.chat_service import ChatService
from gerd.models.model import PromptConfig

chat = ChatService(load_gen_config("hello"))

/Users/aneumann9/micromamba/envs/gerd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
res = chat.submit_user_message({"word": "teleportation"})
res.text

'Sure thing, buddy! Here they are: teleportation, teleportation, teleportation.'

In [5]:
chat.set_prompt_config(PromptConfig.model_validate({"text": "{message}"}))
res = chat.submit_user_message({"message": "Hello! What is one plus one?"})
res.text

'Sure thing, buddy! One plus one equals two.'